In [16]:
%pip install sshtunnel

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import os
import psycopg2
from sshtunnel import SSHTunnelForwarder
from dotenv import load_dotenv

In [11]:
# Load environment variables from .env file
load_dotenv("environment_variables.env")
api_key = os.getenv("API_KEY")
psql_pw = os.getenv("PSQL_PW")

In [29]:
# SSH parameters
bastion_host = 'ec2-34-224-93-62.compute-1.amazonaws.com'
bastion_user = 'ec2-user'
bastion_key = 'C:\\Users\\detto\\Documents\\ec2-key-pair.pem'

# RDS parameters
rds_host = 'youtubeviewprediction.cd0c8oow2pnr.us-east-1.rds.amazonaws.com'
rds_user = 'postgres'
rds_password = os.getenv("PSQL_PW")
rds_database = 'YouTubeViewPrediction'
rds_port = 5432

# Create an SSH tunnel
with SSHTunnelForwarder(
    (bastion_host, 22),
    ssh_username=bastion_user,
    ssh_pkey=bastion_key,
    remote_bind_address=(rds_host, rds_port),
    local_bind_address=('localhost', 6543)  # Choose a local port for the tunnel
) as tunnel:
    # Connect to PostgreSQL through the tunnel
    conn = psycopg2.connect(
        database=rds_database,
        user=rds_user,
        password=rds_password,
        host=tunnel.local_bind_host,
        port=tunnel.local_bind_port
    )

    # Create a cursor object using the cursor() method
    cursor = conn.cursor()

    # Execute a SQL query
    cursor.execute("SELECT version();")

    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

You are connected to -  ('PostgreSQL 16.1 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-12), 64-bit',) 

